In [1]:
from nuplan.database.nuplan_db_orm.nuplandb import NuPlanDB
from pathlib import Path
from nuplan.database.nuplan_db_orm.nuplandb_wrapper import NuPlanDBWrapper
from nuplan.database.nuplan_db_orm.frame import Frame
import os

from nuplan.planning.utils.multithreading.worker_pool import Task
from nuplan.planning.utils.multithreading.worker_parallel import SingleMachineParallelExecutor
from nuplan.planning.scenario_builder.scenario_filter import ScenarioFilter
from nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_builder import NuPlanScenarioBuilder
from nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_utils import ScenarioMapping

import os
import math
import argparse
import matplotlib.pyplot as plt
from tqdm import tqdm
from data_utils import *
from trajectory_tree_planner import *
from common_utils import get_filter_parameters, get_scenario_map
from data_process import DataProcessor
from train_utils import DrivingData

In [2]:
db_path = "/cailiu2/Diffusion-Planner/data/2021.10.21.14.43.30_veh-28_01244_01519.db" # single db file
db_path = "/share/data_cold/open_data/nuplan/data/cache/mini"  # multi db files
data_root = "/cailiu2/Diffusion-Planner/data/"
map_path = "/cailiu2/Diffusion-Planner/data/maps"
map_version = "nuplan-maps-v1.0"

save_processed_path = "/cailiu2/Diffusion-Planner/data/processed"

In [ ]:
path = ["/cailiu2/Diffusion-Planner/data/processed/us-nv-las-vegas-strip_4c8281241cca5fe0.npz"] # the path must be list
train_set = DrivingData(path, 10, 10)
print("len train set: ", len(train_set))

ego, neighbors, map_lanes, map_crosswalks, route_lanes, ego_future_gt, neighbors_future_gt, first_stage, second_stage = train_set[0]


len train set:  1


In [9]:
print("len ego: ", len(ego))
print("len neighbors: ", len(neighbors))
print("len map_lanes: ", len(map_lanes))
print("len map_crosswalks: ", len(map_crosswalks))
print("len route_lanes: ", len(route_lanes))
print("len ego_future_gt: ", len(ego_future_gt))
print("len route_lanes: ", len(route_lanes))
print("len ego_future_gt: ", len(ego_future_gt))
print("len neighbors_future_gt: ", len(neighbors_future_gt))
print("len first_stage: ", len(first_stage))
print("len second_stage: ", len(second_stage))



len ego:  21
len neighbors:  20
len map_lanes:  40
len map_crosswalks:  5
len route_lanes:  10
len ego_future_gt:  80
len route_lanes:  10
len ego_future_gt:  80
len neighbors_future_gt:  10
len first_stage:  10
len second_stage:  10


In [10]:
neighbor = neighbors[0]
print("neighbor: ", neighbor)

neighbor:  [[-2.5073505e+01 -3.5098851e+00  1.1155486e-02  1.2125978e+01
   7.0382878e-02  6.4110663e-04  5.0404859e+00  1.8838710e+00
   1.0000000e+00  0.0000000e+00  0.0000000e+00]
 [-2.3823746e+01 -3.5343525e+00  1.1796593e-02  1.2194538e+01
   7.4816376e-02  4.7311592e-03  5.0266089e+00  1.8714266e+00
   1.0000000e+00  0.0000000e+00  0.0000000e+00]
 [-2.2572762e+01 -3.4963319e+00  1.2101531e-02  1.2244821e+01
   7.9095379e-02 -5.5693565e-03  5.0320187e+00  1.8758632e+00
   1.0000000e+00  0.0000000e+00  0.0000000e+00]
 [-2.1323002e+01 -3.5207994e+00  1.0682941e-02  1.2266829e+01
   6.7682728e-02 -9.2757232e-03  5.0132909e+00  1.8791789e+00
   1.0000000e+00  0.0000000e+00  0.0000000e+00]
 [-2.0072018e+01 -3.4827788e+00  1.0246753e-02  1.2331821e+01
   7.5946391e-02  5.7935189e-03  5.0183420e+00  1.8879956e+00
   1.0000000e+00  0.0000000e+00  0.0000000e+00]
 [-1.8822256e+01 -3.5072463e+00  1.1841416e-02  1.2326235e+01
   8.0032483e-02  1.2122999e-02  5.0664811e+00  1.8855366e+00
   1.